In [1]:
# Author: Albert de Jamblinne de Meux
# thealbertsmail@gmail.com
# All right reserved.
#
# Under Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International Public License
# Read the LICENSE.TXT for detail or https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode


# Extraction of the issuer Name, Method used during the diHub Hackaton by the Mighty python tream for
# the final prediction.

# This is the refactored version. with some simplifications and generalisation and a lot more comments.

# You need the ordered_set and html2text library... (pip install ordered_set html2text)

# Training was done manually, improvement could probably be obtained by further training
# and atomisation of this training which was not done due to the lack of time
# (and because training the model is not straightforward... or at least for me, if
# you see a method to do it, I'm will be more than happy to learn it ;) )
#
# Notice that all the data sets and reference data are confidential. Therefore I have
# not included any of those here, to use this code you will obviously need these or similar
# one... (Notice that I do not have anymore access to those data myself so there is no point asking me for them !!!)

# This code only extract the values.

Create the model
-----------------

In [ ]:

# First we create the model and save it.
from NEModel import matcher,extractor
import ordered_set

# create the extraction model,
with open('../../company.csv','r') as f: # list of all the companies from which we have to choOse from.
    c = f.readlines()
    
companies =  ordered_set.OrderedSet()

for item in c[1:]:
    companies.add(item.strip('\n", '))

# some synonymes
synonymes =[('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate 110'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate 110'),
    ('london','bishopsgate 110'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate 110'),
    ('rubbermaid inc','brands inc'),
    ('dexia funding netherlands','belfius financing company sa'),
    ('province of british columbia','government of the province of british columbia'),
    ('london branch','one cabot square')]

match = matcher(companies,synonymes)

extract = extractor(match, ('issue','issued'), ('issuer','issued'),
                    ( 'linked to', 'share issuer', 'equity issuer', 'guarantor', 'dealer',
                      'of the issue of securities', 'lead manager','notes and certificates of'),
                    (('BARCLAYS BANK PLC Incorporated with limited liability in england and wales',
                      'BARCLAYS BANK PLC','BARCLAYS BANK PLC BISHOPSGATE (110)'),),
                    start='+')

extract.save('extractor.pkl')

Use it in serial
-----------------

In [3]:
#%%timeit

# Now we can extract the data
import os
import Issuer_extraction.txt_processing as tp
import Issuer_extraction.label_data as de
from NEModel import matcher,extractor
import ordered_set

folder = '../../Newdata/train' # where are the data...
# get the files (this is mapping fileid to file path.)
filesTxT = tp.get_files(os.path.join(folder,'txt')) # txt files
filesHtml = tp.get_files(os.path.join(folder,'html')) # html files

# get the fileids for the data set (here for the training set...):
d = de.data(folder='../../Newdata/train/')

# get the extractor
extract=extractor.from_file('extractor.pkl')

# where to store the data.
issuerDic = {}

x = 0
for isin in d.docid:  # for each isin in the data object created before
   
    if x > 100: # to limit the number of document to check
        break
    x += 1

    f = d.docid[isin]  # get the file_ids, this is a set, because we can have many document.
    issuer = None # the issuer, at the start we do not known...
    for item in f:  # for each of these files
        # get the txt from the file first.
        txt = tp.get_text(filesTxT[item], raw=True)
        #print txt
        issuer = extract.extract(txt) # get the value
        if issuer is None: # if it did not work, try the html file,
            # I never test if this is really useful, but when I started, some txt files where bad
            # while the html files was ok. Nevertheless I could not find a nice why to striping html tag,
            # ofter I lose most of the structure, the one provided keep the structures which is much better...
            txt = tp.get_text(filesHtml[item])
            issuer = extract.extract(txt)

        # if we find it, stop searching...
        if issuer is not None:
            break
            
    #print isin, issuer

    if issuer is not None:
        issuerDic[isin] = issuer
    else:
        issuerDic[isin] = '' # put an empty string for isin that where not found...

p = {}
for item in issuerDic:
    p[item]=d.issuer[item][0]
de.test(issuerDic,p)
        
# save the data in csv...
de.to_csv('issuer.csv',issuerDic)

// version:
--------------

This is a fast solution, no the best one ;)

In [3]:

import Issuer_extraction.txt_processing as tp
import Issuer_extraction.label_data as de
from NEModel import matcher,extractor

import os
import ordered_set
from multiprocessing import Pool
from functools import partial

d = de.data(folder='../../Newdata/train/')
folder = '../../Newdata/train' # where are the data...
# get the extractor
extract=extractor.from_file('extractor.pkl')

In [ ]:
#%%time
# Or we can try to do this in //

def get_issuer(isins, extract, folder):
    """
    Extract the issuer of a set of isin from a given folder.
    
    Args:
        extract: the extractor use to get the issuer
        folder: the path to the root folder (only train / final_test and / int_test folder)
        isins: list of isin to be extracted.
    """
    
    # get the files (this is mapping fileid to file path.)
    filesTxT = tp.get_files(os.path.join(folder,'txt')) # txt files
    filesHtml = tp.get_files(os.path.join(folder,'html')) # html files

    # get the fileids for the data set (here for the training set...):
    d = de.data(folder=folder)
    
    # where to store the data.
    issuerDic = {}
    
    for isin in isins:  # for each isin in the data object created before

        f = d.docid[isin]  # get the file_ids, this is a set, because we can have many document.
        issuer = None # the issuer, at the start we do not known...
        for item in f:  # for each of these files
            # get the txt from the file first.
            txt = tp.get_text(filesTxT[item], raw=True)
            #print txt
            issuer = extract.extract(txt) # get the value
            if issuer is None: # if it did not work, try the html file,
                # I never test if this is really useful, but when I started, some txt files where bad
                # while the html files was ok. Nevertheless I could not find a nice why to striping html tag,
                # ofter I lose most of the structure, the one provided keep the structures which is much better...
                txt = tp.get_text(filesHtml[item])
                issuer = extract.extract(txt)

            # if we find it, stop searching...
            if issuer is not None:
                break

        if issuer is not None:
            issuerDic[isin] = issuer
        else:
            issuerDic[isin] = '' # put an empty string for isin that where not found...

    return issuerDic


def get_issuer_multi(extract, folder, isins, ncpu):
    """
    Extract the issuer of a set of isin from a given folder.
    
    multi process version.
    
    Args:
        extract: the extractor use to get the issuer
        folder: the path to the root folder (only train / final_test and / int_test folder)
        isins: list of isin to be extracted.
        ncpu: use ncpu core.
    """
    l = len(isins) / ncpu
    groups = []
    for x in xrange(0,ncpu-1):
        groups.append(isins[x*l:(x+1)*l])
    groups.append(isins[(x+1)*l:])
    
    p = Pool(ncpu)
        
    f = partial(get_issuer, extract=extract, folder=folder)
        
    result = p.map(f, groups)

    rf={}
    for r in result:
        rf.update(r)
        
    return rf

issuerDic = get_issuer_multi(extract, folder, d.docid.keys()[:101], 2)

p = {}
for item in issuerDic:
    p[item]=d.issuer[item][0]
de.test(issuerDic,p)

de.to_csv('issuer.csv',issuerDic)